<a href="https://colab.research.google.com/github/fwkhan/CE888/blob/main/tweetAssignment/baseline_model_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install zeugma

  Created wheel for zeugma: filename=zeugma-0.48-cp37-none-any.whl size=8778 sha256=0c3db20acda7be7e6ee4857fda415ee1b7d3a3a662ba7631c7c1eda709266753
  Stored in directory: /root/.cache/pip/wheels/6a/b5/bc/5183ac478b0071d04d3ed0c0dd4a43db94c5c8ffb317b5eb53
Successfully built zeugma


In [2]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
# import torch
# if torch.cuda.is_available():       
#     device = torch.device("cuda")
#     print(f'There are {torch.cuda.device_count()} GPU(s) available.')
#     print('Device name:', torch.cuda.get_device_name(0))

# else:
#     print('No GPU available, using the CPU instead.')
#     device = torch.device("cpu")


In [4]:
import numpy as np
import pandas as pd
# Marry nltk and ScikitLearn
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
import pickle
from nltk import word_tokenize
import nltk
import random
from nltk.corpus import stopwords
from nltk.classify import ClassifierI
from statistics import mode
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import seaborn as sns
import requests

import warnings
warnings.filterwarnings('ignore')

# Preprocessing

In [5]:
############################ SENTIMENT ANALYSIS #################################################
SENTIMENT_TRAIN_TEXT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/train_text.txt'
SENTIMENT_VALIDATION_TEXT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/val_text.txt'
SENTIMENT_TEST_TEXT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/test_text.txt'

SENTIMENT_TRAIN_LABEL = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/train_labels.txt'
SENTIMENT_VALIDATION_LABEL = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/val_labels.txt'
SENTIMENT_TEST_LABEL = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/test_labels.txt'

############################ HATE #################################################
HATE_TRAIN_TEXT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/train_text.txt'
HATE_VALIDATION_TEXT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/val_text.txt'
HATE_TEST_TEXT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/test_text.txt'

HATE_TRAIN_LABEL = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/train_labels.txt'
HATE_VALIDATION_LABEL = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/val_labels.txt'
HATE_TEST_LABEL = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/test_labels.txt'

############################ OFFENSIVE LANGUAGE#################################################
OFFENSE_TRAIN_TEXT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/train_text.txt'
OFFENSE_VALIDATION_TEXT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/val_text.txt'
OFFENSE_TEST_TEXT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/test_text.txt'

OFFENSE_TRAIN_LABEL = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/train_labels.txt'
OFFENSE_VALIDATION_LABEL = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/val_labels.txt'
OFFENSE_TEST_LABEL = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/test_labels.txt'

############################ IRONY #################################################
IRONY_TRAIN_TEXT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/train_text.txt'
IRONY_VALIDATION_TEXT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/val_text.txt'
IRONY_TEST_TEXT = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/test_text.txt'

IRONY_TRAIN_LABEL = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/train_labels.txt'
IRONY_VALIDATION_LABEL = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/val_labels.txt'
IRONY_TEST_LABEL = 'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/test_labels.txt'

In [40]:
def preprocess(df): 
    lemmatizer  = WordNetLemmatizer()
    
    ps = PorterStemmer()
    ignore_words = ['user', 'st'] 
    df['processed_tweets'] = df['tweet'].replace('[^a-zA-Z]',' ', regex=True,
                                                  inplace=False)
    df['processed_tweets'] = df['processed_tweets'].apply(lambda x: [w.lower() for w in x.split()])
    # df['processed_tweets'] = df['processed_tweets'].apply(lambda tweet: ([word for word in tweet if not word in stopwords.words("english")]))
    # df['processed_tweets'] = df['processed_tweets'].apply(lambda tweet: ([lemmatizer.lemmatize(word) for word in tweet]))
    # df['processed_tweets'] = df['processed_tweets'].apply(lambda tweet: ([ps.stem(word) for word in tweet]))

 
 
    df['processed_tweets'] = df['processed_tweets'].apply(lambda tweet: ' '.join([word for word in tweet if len(word)>2]))
    # df['processed_tweets'] = df['processed_tweets'].apply(lambda tweet: ' '.join([word for word in tweet]))

 
    df['processed_tweets'] = df['processed_tweets'].apply(lambda x: ' '.join([word for word in x.split() if not word in ignore_words]))
    
    df["sentence_length"] = df.tweet.apply(lambda x: len(str(x).split()))
    return df
 
 
# Wrapper to convert text data to pandas Dataframe
def txt_to_df(data, label, classification_task):
    tweet = []
    sentiments = []
    for sentence in data.split('\n'):
        tweet.append(sentence)
    for sentiment in label.split('\n'):
        try:
            sentiments.append(int(sentiment))
        except ValueError:
            pass
    df= pd.DataFrame(tweet[:-1], columns=['tweet'])
    df['label'] = sentiments
    if classification_task == 'Sentiment_analysis':
      df['sentiment'] = df.label.apply(lambda x: 'Negative'if x==0 else ('Neutral' if x==1 else 'Positive'))
    if classification_task == 'hate_analysis':
      df['sentiment'] = df.label.apply(lambda x: 'Not-hate'if x==0 else 'hate')
    if classification_task == 'offensive_analysis':
      df['sentiment'] = df.label.apply(lambda x: 'Not-offensive 'if x==0 else 'offensive')
    if classification_task == 'irony_analysis':
      df['sentiment'] = df.label.apply(lambda x: 'Non-irony 'if x==0 else 'irony')      
    return df
 
 
def prepare_dataset(TRAIN_TEXT, TRAIN_LABEL, VAL_TEXT, VAL_LABEL, TEST_TEXT, TEST_LABEL, classification_task):
  # Reading Train, Vvalidation & Test data from tweeteval Github Repo.
  train_tweets_txt = requests.get(TRAIN_TEXT).text
  train_labels_txt = requests.get(TRAIN_LABEL).text
 
  val_tweets_txt = requests.get(VAL_TEXT).text
  val_labels_txt = requests.get(VAL_LABEL).text
 
  test_tweets_txt = requests.get(TEST_TEXT).text
  test_labels_txt = requests.get(TEST_LABEL).text
 
  # Converting text data to pandas Dataframe
  train_df = txt_to_df(train_tweets_txt, train_labels_txt, classification_task)
  val_df = txt_to_df(val_tweets_txt, val_labels_txt, classification_task)
  test_df = txt_to_df(test_tweets_txt, test_labels_txt, classification_task)
 
  train_df = preprocess(train_df)
  val_df = preprocess(val_df)
  test_df = preprocess(test_df)  
 
  return train_df, val_df, test_df

#Random OverSampling

In [7]:
from imblearn.over_sampling import RandomOverSampler

def smot_oversmapling(X, y):
  X = X.reshape(-1,1)
  oversample = RandomOverSampler()

  X_train_smot, y_train_smot = oversample.fit_resample(X, y)
  X_train_smot = X_train_smot.reshape(len(X_train_smot),)
  # print('Shape of Training and Labelled data after random oversampling:', X_train_smot.shape, y_train_smot.shape)
  return X_train_smot, y_train_smot

# Feature Extraction







In [8]:
def print_embedding_shape(X_train, y_train, X_val, y_val, X_test, y_test):
  print('Shape of Training and Labelled data after random oversampling:', X_train.shape, y_train.shape)
  # print('Shape of Validation and Labelled data after random oversampling:', X_val.shape, y_val.shape)
  # print('Shape of Test and Labelled data after random oversampling:', X_test.shape, y_test.shape)

##Word Embedding
---Tranforming Text to Vectors


###Count Vectorizer

#### Total features

*   n-gram - 37221
*   bi-grams - 328966
*   tri-grams - 682076





In [9]:
# Transformin Dataset into a vector on the basis of the frequency (count) of each word that occurs in the entire text.
from sklearn.feature_extraction.text import CountVectorizer

def embedding_count_vectorize(X_train, y_train, X_val, y_val, X_test, y_test, ngram=1):
  # Create Vector object
  countvector=CountVectorizer(ngram_range=(1,ngram)) # max_features = default, ngrams_range=(1,1)

  # Fit and Transform Train data
  X_train=countvector.fit_transform(X_train)

  # Printing the identified Unique words along with trigrams 
  vocab_dict = countvector.vocabulary_
  # print('Sample Vocabulary',{k: vocab_dict[k] for k in list(vocab_dict)[:20]})

  # Summarizing the Encoded Texts 
  # print("Encoded Document is:") 
  # print(countvector.toarray())

  # Transform Validation dataset
  X_val = countvector.transform(val_df['processed_tweets'])
  y_val = val_df.label

  # Transform Test dataset
  X_test = countvector.transform(test_df['processed_tweets'])
  y_test = test_df.label

  print_embedding_shape(X_train, y_train, X_val, y_val, X_test, y_test)
  return X_train, y_train, X_val, y_val, X_test, y_test

###TF-IDF

#### Total features

*   n-gram - 37221
*   bi-grams - 328966
*   tri-grams - 682076


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

def embedding_tfidf(X_train, y_train, X_val, y_val, X_test, y_test, ngram=1):

  tokenization = TfidfVectorizer(ngram_range=(1,ngram)) # max_features=6135
  X_train = tokenization.fit_transform(X_train)
  # X_train = X_train.toarray()

  X_val = tokenization.transform(val_df['processed_tweets'])
  y_val = val_df.label

  # Transform Test dataset
  X_test = tokenization.transform(test_df['processed_tweets'])
  y_test = test_df.label
  print_embedding_shape(X_train, y_train, X_val, y_val, X_test, y_test)
  return X_train, y_train, X_val, y_val, X_test, y_test

In [11]:
from gensim.models import Word2Vec
w2v_model = 0 
def embedding_Word2Vec(X_train, y_train, X_val, y_val, X_test, y_test):

  X_train = pd.DataFrame(X_train, columns=['tweet'])
  X_train['tokend'] = X_train['tweet'].apply(word_tokenize)

  X_val = pd.DataFrame(X_val, columns=['tweet'])
  X_val['tokend'] = X_val['tweet'].apply(word_tokenize)

  X_test = pd.DataFrame(X_test, columns=['tweet'])
  X_test['tokend'] = X_test['tweet'].apply(word_tokenize)


  # w2v = Word2Vec(X_train.tokend, size=350, window=10, min_count=1, iter=20)

  def text_to_vector(text):
      """Create document vectors by averaging word vectors. Remove out-of-vocabulary words."""
      text = [word for word in text if word in w2v_model.wv.vocab]
      if not len(text):
        text = ['tweet']
      return np.mean(w2v_model[text], axis=0)


  w2v_model = Word2Vec(X_train.tokend, min_count=20,
                      window=7,
                      size=300,
                      sample=6e-5, 
                      alpha=0.03, 
                      min_alpha=0.0007, 
                      negative=10)

  X_train['doc_vector'] = X_train['tokend'].apply(text_to_vector)
  X_val['doc_vector'] = X_val['tokend'].apply(text_to_vector)
  X_test['doc_vector'] = X_test['tokend'].apply(text_to_vector)

  x_train = list(X_train['doc_vector'])
  x_val = list(X_val['doc_vector'])
  x_test = list(X_test['doc_vector'])
  train_model(x_train, y_train, x_val, y_val, x_test, y_test)

In [12]:
# pip install zeugma

In [33]:
from zeugma.embeddings import EmbeddingTransformer

def embedding_Glove(X_train, y_train, X_val, y_val, X_test, y_test):
    glove = EmbeddingTransformer('glove')
    x_train = glove.transform(X_train)
    x_val = glove.transform(X_val)
    x_test = glove.transform(X_test)
    
    train_model(x_train, y_train, x_val, y_val, x_test, y_test)


#Get Prepared Datasets

In [14]:
def get_dataset():
  X = train_df.processed_tweets.values
  y = train_df.label.values

  X_train, y_train = smot_oversmapling(X, y)

  X_val = val_df.processed_tweets.values
  y_val = val_df.label.values

  X_test = test_df.processed_tweets.values
  y_test = test_df.label.values
  return X_train, y_train, X_val, y_val, X_test, y_test

# Classification

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
def logistic_classifier(X_train, y_train):
  """ 
    Initialize LogisticRegression Object. 

    This function initializes the parameters on which Grid Search has to be 
    performed and calls the api to Grid Search, which returns the best
    estimator for LogisticRegression. 

    Parameters: 
    X_train (DataFrame): Input Training Data set  
    y_train (DataFrame): Labelled Training Data 
    X_test (DataFrame): Input Validation Data set
    y_test (DataFrame): Labelled Validation Data


    Returns: 
    classifier_lr: Best esitmator for LogisticRegression 

  """    
  model = LogisticRegression(max_iter=100)
  # model = LinearSVC(C=0.5, random_state=70, max_iter=2000)

  return model

In [16]:
from sklearn.metrics import recall_score, f1_score

def fit_models(classifier, X_train, y_train, X_val, y_val, X_test, y_test):
    """ 
      Wrapper to train and Validate Classifiers. 

      This is a wrapper function which trains the classifer, gets the test 
      score, performs k-fold cross validation with 10 splits.

      Parameters: 
      X_train (DataFrame): Input Training Data set  
      y_train (DataFrame): Labelled Training Data 
      X_test (DataFrame): Input Validation Data set
      y_test (DataFrame): Labelled Validation Data

      Returns: 
      test_score: Score of prediction done on test data
      accuracy_mean: Average Cross Validation Score
      cross_valScore: List of all the Cross Validation Score

    """     
    classifier.fit(X_train, y_train)

    validation_score = classifier.score(X_val,y_val)
    test_score = classifier.score(X_test,y_test)

    y_val_pred = classifier.predict(X_val)
    recallScore_val = recall_score(y_val, y_val_pred, average='macro')
    f1Score_val = f1_score(y_val, y_val_pred, average='macro')

    y_test_pred = classifier.predict(X_test)
    recallScore_test = recall_score(y_test, y_test_pred, average='macro')
    f1Score_test = f1_score(y_test, y_test_pred, average='macro')


    return validation_score, recallScore_val, test_score, recallScore_test, f1Score_val, f1Score_test

In [17]:
def train_model(X_train, y_train, X_val, y_val, X_test, y_test):
  classifier_lr = logistic_classifier(X_train, y_train)
  validation_score, recallScore_val, test_score, recallScore_test, f1Score_val, f1Score_test = fit_models(classifier_lr, X_train, y_train, X_val, y_val, X_test, y_test)
  print('Validation Score: {}\nTest Score: {}'
    .format(validation_score,test_score)) 
  print('Validation Recall Score: {}\nTest Recall Score: {}'
    .format(recallScore_val, recallScore_test)) 
  print('Validation F1 Score: {}\nTest F1 Score: {}'
    .format(f1Score_val, f1Score_test)) 

In [18]:
def embedding_count_vectorize_ngram(X_train, y_train, X_val, y_val, X_test, y_test):
  for ngram in range(1,4):
    print('N-GRAM: {}'.format(ngram))
    X_train, y_train, X_val, y_val, X_test, y_test = get_dataset()
    X_train, y_train, X_val, y_val, X_test, y_test = embedding_count_vectorize(X_train, y_train, X_val, y_val, X_test, y_test, ngram)
    train_model(X_train, y_train, X_val, y_val, X_test, y_test)

In [19]:
def embedding_tfid_ngram(X_train, y_train, X_val, y_val, X_test, y_test):
  for ngram in range(1,4):
    print('N-GRAM: {}'.format(ngram))
    X_train, y_train, X_val, y_val, X_test, y_test = get_dataset()
    X_train, y_train, X_val, y_val, X_test, y_test = embedding_tfidf(X_train, y_train, X_val, y_val, X_test, y_test, ngram)
    train_model(X_train, y_train, X_val, y_val, X_test, y_test)

In [20]:
def init_training(features_extraction):
  X_train, y_train, X_val, y_val, X_test, y_test = get_dataset()
  feature_extraction(X_train, y_train, X_val, y_val, X_test, y_test)
  # train_model(X_train, y_train, X_val, y_val, X_test, y_test)  

In [41]:
import warnings
warnings.filterwarnings('ignore')
# feature_extraction_strategy = [embedding_tfid_ngram, embedding_count_vectorize_ngram, embedding_Glove, embedding_Word2Vec]
feature_extraction_strategy = [embedding_Glove]
classification_task = {'SENTIMENT_ANALYSIS' : 'Sentiment_analysis',
                       'HATE_ANALYSIS' : 'hate_analysis',
                       'OFFENSIVE_LANGUAGE' : 'offensive_analysis',
                       'IRONY_ANALYSIS' : 'irony_analysis'
                       }

for key, task in classification_task.items():
  print('=========================================')
  print('CLASSIFICATION TASK: {}'.format(key))
  print('=========================================')
  if key == 'SENTIMENT_ANALYSIS':
    continue
    train_df, val_df, test_df = prepare_dataset(SENTIMENT_TRAIN_TEXT, SENTIMENT_TRAIN_LABEL,
                        SENTIMENT_VALIDATION_TEXT, SENTIMENT_VALIDATION_LABEL,
                        SENTIMENT_TEST_TEXT, SENTIMENT_TEST_LABEL, classification_task['SENTIMENT_ANALYSIS']
                        )
  if key == 'HATE_ANALYSIS':
    # continue
    train_df, val_df, test_df = prepare_dataset(HATE_TRAIN_TEXT, HATE_TRAIN_LABEL,
                        HATE_VALIDATION_TEXT, HATE_VALIDATION_LABEL,
                        HATE_TEST_TEXT, HATE_TEST_LABEL, classification_task['HATE_ANALYSIS']
                        )
  if key == 'OFFENSIVE_LANGUAGE':
    continue
    train_df, val_df, test_df = prepare_dataset(OFFENSE_TRAIN_TEXT, OFFENSE_TRAIN_LABEL,
                        OFFENSE_VALIDATION_TEXT, OFFENSE_VALIDATION_LABEL,
                        OFFENSE_TEST_TEXT, OFFENSE_TEST_LABEL, classification_task['OFFENSIVE_LANGUAGE']
                        )
  if key == 'IRONY_ANALYSIS':
    continue
    train_df, val_df, test_df = prepare_dataset(OFFENSE_TRAIN_TEXT, OFFENSE_TRAIN_LABEL,
                        OFFENSE_VALIDATION_TEXT, OFFENSE_VALIDATION_LABEL,
                        OFFENSE_TEST_TEXT, OFFENSE_TEST_LABEL, classification_task['IRONY_ANALYSIS']
                        )    
  for feature_extraction in feature_extraction_strategy:
    print('FEATURE EXTRACTION STRATEGY TASK: {}'.format(feature_extraction))
    init_training(feature_extraction)

CLASSIFICATION TASK: SENTIMENT_ANALYSIS
CLASSIFICATION TASK: HATE_ANALYSIS
FEATURE EXTRACTION STRATEGY TASK: <function embedding_Glove at 0x7f8e97f42c20>
Validation Score: 0.651
Test Score: 0.5595959595959596
Validation Recall Score: 0.6396691884203686
Test Recall Score: 0.5865023413062964
Validation F1 Score: 0.6404874948365145
Test F1 Score: 0.5567975371708405
CLASSIFICATION TASK: OFFENSIVE_LANGUAGE
CLASSIFICATION TASK: IRONY_ANALYSIS
